## Section 1. Introduction ##

In this notebook, the dataset to be processed is the Labor Force Survey conducted April 2016 and retrieved through Philippine Statistics Authority database. 



In [ ]:
import os
import random
import pickle
import numpy as np
import pandas as pd
import h5py


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

plt.rcParams['figure.figsize'] = (6.0, 6.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

plt.style.use('ggplot')

# autoreload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

<h1>Importing LFS PUF April 2016.CSV</h1>

In [ ]:
try:
    lfs_data = pd.read_csv("LFS PUF April 2016.CSV")
except FileNotFoundError:
    print("Error: CSV file not found. Please make sure the file exists in the correct directory or provide the correct path.")
    exit()

<h1>Data Information, Pre-Processing, and Cleaning</h1>

Let's get an overview of our dataset.

In [ ]:
lfs_data.info()

---
Of interest to us, there are:
<ul><li>1 contains float values, </li>
<li>14 contain integer values, and </li>
<li><b>35 are object values</b>.</li></ul>


---
Let's check for duplicates:

In [ ]:
lfs_data.duplicated().sum()

No duplicates here, and therefore no cleaning need follow in this regard.

The dataset seems to contain null values in the form of whitespaces. Let's count those:

In [ ]:
has_null = lfs_data.apply(lambda col: col.str.isspace().sum() if col.dtype == 'object' else 0)

print("Number Empty Cells:")
print(has_null[has_null > 0])

---
And standardize, replacing these whitespace values with NaN:

In [ ]:
lfs_data.replace(r"^\s+$", np.nan, regex=True, inplace=True)
nan_counts_per_column = lfs_data.isna().sum()
print(nan_counts_per_column[nan_counts_per_column > 0])

Now that these are NaN, let's return to the data types, and find if our object columns from earlier are convertible to integers (or float):

In [ ]:
int_convertible_columns = []

for col in lfs_data.columns:
    if lfs_data[col].dtypes == 'object':  
        try:
            float_vals = lfs_data[col].dropna().astype(float)
            if (float_vals % 1 == 0).all():
                int_convertible_columns.append(col)
        except ValueError:
            pass 

print("Safely convertable to int:")
print(int_convertible_columns)

---
And convert to pd.Int32Dtype, as to handle potential NaNs

In [ ]:
columns_to_convert = [
    'PUFC06_MSTAT', 'PUFC08_CURSCH', 'PUFC09_GRADTECH', 'PUFC10_CONWR', 'PUFC11_WORK', 
    'PUFC12_JOB', 'PUFC14_PROCC', 'PUFC16_PKB', 'PUFC17_NATEM', 'PUFC18_PNWHRS', 
    'PUFC19_PHOURS', 'PUFC20_PWMORE', 'PUFC21_PLADDW', 'PUFC22_PFWRK', 'PUFC23_PCLASS', 
    'PUFC24_PBASIS', 'PUFC25_PBASIC', 'PUFC26_OJOB', 'PUFC27_NJOBS', 'PUFC28_THOURS', 
    'PUFC29_WWM48H', 'PUFC30_LOOKW', 'PUFC31_FLWRK', 'PUFC32_JOBSM', 'PUFC33_WEEKS', 
    'PUFC34_WYNOT', 'PUFC35_LTLOOKW', 'PUFC36_AVAIL', 'PUFC37_WILLING', 'PUFC38_PREVJOB', 
    'PUFC40_POCC', 'PUFC41_WQTR', 'PUFC43_QKB', 'PUFNEWEMPSTAT'
]

for col in columns_to_convert:
    if col in lfs_data.columns: 
        try:
            lfs_data[col] = lfs_data[col].astype(pd.Int64Dtype())
        except ValueError:
            print(f"Could not convert column {col} to nullable integer due to invalid values.")

print("Conversion complete.")


---
Let's also apply the unique() function to our dataset.

In [ ]:
lfs_data.apply(lambda x: x.nunique())

---
Considering our dataset has 18,000 entries, features with particularly low numbers stand out as questions that have clear, defined choices. Reviewing the [questionnaire](https://psada.psa.gov.ph/catalog/67/download/537), we find that certain questions ask the participant to specify beyond prespecified choices.

This column possibly contains "010," which is obviously not an integer. We ensure this column is a string, and check for values not specified in the questionnaire.

In [ ]:
lfs_data['PUFC07_GRADE'] = lfs_data['PUFC07_GRADE']
valid_codes = [
    0, 10,  # No Grade, Preschool
    210, 220, 230, 240, 250, 260, 280,  # Elementary
    310, 320, 330, 340, 350,  # High School
    410, 420,  # Post Secondary; If Graduate Specify
    810, 820, 830, 840,  # College; If Graduate Specify
    900,  # Post Baccalaureate
    np.nan
]
invalid_rows = lfs_data[~(lfs_data['PUFC07_GRADE'].isin(valid_codes))]

unique_invalid_values = invalid_rows['PUFC07_GRADE'].unique()
print(unique_invalid_values)

Values 5XX 6XX are not detailed in the questionnaire. As it instructs the participant to specify whether they graduated from post secondary or college, we'll create a new data point to encapsulate these.

In [ ]:
lfs_data.loc[~lfs_data['PUFC07_GRADE'].isin(valid_codes), 'PUFC07_GRADE'] = 700
print(lfs_data['PUFC07_GRADE'].unique())

## EDA

In [ ]:
corr_matrix = lfs_data.corr()

strong_correlations = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)): 
        corr_value = corr_matrix.iloc[i, j]
        if (0.5 < corr_value < 1) or (-1 < corr_value < -0.5):
            strong_correlations.append((
                corr_matrix.index[i], 
                corr_matrix.columns[j], 
                corr_value
            ))

strong_correlations.sort(key=lambda x: abs(x[2]), reverse=True)

print("Strong correlations (|corr| > 0.5 and |corr| < 1):")
for var1, var2, corr in strong_correlations:
    print(f"{var1} — {var2}: {corr:.3f}")

# Regression

In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

class LFSDataset(Dataset):
    def __init__(self, features, labels, missing_value=-1):
        self.features = features.values.astype(np.float32)
        
        unique_labels = np.unique(labels)
        if len(unique_labels) <= 2 and 0 in unique_labels and 1 in unique_labels:
            self.labels = labels.values.astype(np.float32)
        else:
            min_label = labels.min()
            self.labels = (labels.values != min_label).astype(np.float32)
            print(f"Normalized target values from {unique_labels} to [0, 1]")
        
        self.missing_value = missing_value
        self.mask = (self.features != missing_value).astype(np.float32)
        self.features = np.where(self.features == missing_value, 0, self.features)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'features': self.features[idx],
            'mask': self.mask[idx],
            'labels': self.labels[idx]
        }

class MaskedLogisticRegression(torch.nn.Module):
    def __init__(self, input_dim):
        super(MaskedLogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, 1)
        
    def forward(self, features, mask):
        masked_features = features * mask
        output = self.linear(masked_features)
        return torch.sigmoid(output)

def prepare_data(lfs_data, target_col='PUFC11_WORK', feature_cols=None, test_size=0.2, missing_value=-1):
    if feature_cols is None:
        feature_cols = [
            'PUFC05_AGE', 'PUFC06_MSTAT', 'PUFC04_SEX', 
            'PUFC07_GRADE', 'PUFC08_CURSCH', 
            'PUFC38_PREVJOB', 'PUFC31_FLWRK',
            'PUFC30_LOOKW', 'PUFC34_WYNOT'
        ]
    
    available_features = [col for col in feature_cols if col in lfs_data.columns]
    if not available_features:
        raise ValueError("None of the specified features are in the dataset")
    
    if target_col not in lfs_data.columns:
        raise ValueError(f"Target column {target_col} not found in dataset")
    
    mask = lfs_data[target_col].notna()
    filtered_data = lfs_data.loc[mask, available_features + [target_col]]
    
    print(f"Target variable: {target_col}")
    print(f"Unique values in target: {sorted(filtered_data[target_col].unique())}")
    
    X = filtered_data[available_features]
    y = filtered_data[target_col]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    train_missing_mask = X_train.isna()
    test_missing_mask = X_test.isna()
    
    X_train_filled = X_train.fillna(0)
    X_test_filled = X_test.fillna(0)
    
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(
        scaler.fit_transform(X_train_filled),
        columns=X_train.columns,
        index=X_train.index
    )
    
    X_test_scaled = pd.DataFrame(
        scaler.transform(X_test_filled),
        columns=X_test.columns,
        index=X_test.index
    )
    
    X_train_scaled[train_missing_mask] = missing_value
    X_test_scaled[test_missing_mask] = missing_value
    
    return {
        'X_train': X_train_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train,
        'y_test': y_test,
        'feature_names': available_features,
        'scaler': scaler
    }

def train_model(data_dict, learning_rate=0.01, batch_size=64, num_epochs=10, 
                scheduler_step_size=3, scheduler_gamma=0.5, missing_value=-1):
    train_dataset = LFSDataset(data_dict['X_train'], data_dict['y_train'], missing_value)
    test_dataset = LFSDataset(data_dict['X_test'], data_dict['y_test'], missing_value)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    input_dim = data_dict['X_train'].shape[1]
    model = MaskedLogisticRegression(input_dim)
    
    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=scheduler_step_size, gamma=scheduler_gamma)
    
    history = {
        'train_loss': [],
        'test_loss': [],
        'train_accuracy': [],
        'test_accuracy': []
    }
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for batch in train_loader:
            features = batch['features']
            mask = batch['mask']
            labels = batch['labels'].view(-1, 1)
            
            outputs = model(features, mask)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            predicted = (outputs >= 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        
        scheduler.step()
        
        avg_train_loss = total_loss / len(train_loader)
        train_accuracy = correct / total
        
        model.eval()
        test_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch in test_loader:
                features = batch['features']
                mask = batch['mask']
                labels = batch['labels'].view(-1, 1)
                
                outputs = model(features, mask)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item()
                
                predicted = (outputs >= 0.5).float()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        
        avg_test_loss = test_loss / len(test_loader)
        test_accuracy = correct / total
        
        history['train_loss'].append(avg_train_loss)
        history['test_loss'].append(avg_test_loss)
        history['train_accuracy'].append(train_accuracy)
        history['test_accuracy'].append(test_accuracy)
        
        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {avg_train_loss:.4f}, '
              f'Test Loss: {avg_test_loss:.4f}, '
              f'Train Acc: {train_accuracy:.4f}, '
              f'Test Acc: {test_accuracy:.4f}, '
              f'LR: {scheduler.get_last_lr()[0]:.6f}')
    
    return {
        'model': model,
        'history': history,
        'feature_names': data_dict['feature_names']
    }

def analyze_model(result_dict):
    history = result_dict['history']
    epochs = range(1, len(history['train_loss']) + 1)
    
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], 'b-', label='Train Loss')
    plt.plot(epochs, history['test_loss'], 'r-', label='Val Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_accuracy'], 'b-', label='Train Acc')
    plt.plot(epochs, history['test_accuracy'], 'r-', label='Val Acc')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    model = result_dict['model']
    feature_names = result_dict['feature_names']
    
    weights = model.linear.weight.data.numpy().flatten()
    bias = model.linear.bias.data.numpy()[0]
    
    coefficients = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': weights
    })
    
    coefficients = coefficients.reindex(coefficients['Coefficient'].abs().sort_values(ascending=False).index)
    
    print(f"\nBias: {bias:.4f}")
    print(coefficients)
    
    plt.figure(figsize=(10, 6))
    plt.barh(coefficients['Feature'], coefficients['Coefficient'].abs(), color='skyblue')
    plt.xlabel('Absolute Coefficient Value')
    plt.title('Feature Importance')
    plt.tight_layout()
    plt.show()

def predict_employment_status(lfs_data, result_dict, target_normalization=None, missing_value=-1):
    model = result_dict['model']
    feature_names = result_dict['feature_names']
    
    X = lfs_data[feature_names]
    mask = (X != missing_value).astype(np.float32)
    X = X.replace(missing_value, 0).astype(np.float32)
    
    model.eval()
    with torch.no_grad():
        predictions = model(torch.tensor(X.values), torch.tensor(mask.values))
        binary_predictions = (predictions >= 0.5).float().numpy().flatten()
    
    if target_normalization is not None and 'original_values' in target_normalization:
        original_values = target_normalization['original_values']
        if len(original_values) == 2:
            value_map = {0: min(original_values), 1: max(original_values)}
            return pd.Series([value_map[int(p)] for p in binary_predictions], index=X.index)
    
    return pd.Series(binary_predictions, index=X.index)

def run_employment_prediction(lfs_data, target_col='PUFC11_WORK'):
    print("Preparing data...")
    data_dict = prepare_data(lfs_data, target_col=target_col)
    
    print(f"Training on {len(data_dict['X_train'])} samples with {len(data_dict['feature_names'])} features...")
    print(f"Features: {data_dict['feature_names']}")
    
    original_values = sorted(lfs_data[target_col].dropna().unique())
    target_normalization = {'original_values': original_values}
    print(f"Original target values: {original_values}")
    
    result_dict = train_model(
        data_dict,
        learning_rate=0.01,
        batch_size=128,
        num_epochs=10,
        scheduler_step_size=3,
        scheduler_gamma=0.5
    )
    
    result_dict['target_normalization'] = target_normalization
    
    print("\nModel training complete!")
    analyze_model(result_dict)
    
    return result_dict

# result_dict = run_employment_prediction(lfs_data)

In [ ]:
result_dict = run_employment_prediction(lfs_data)

WIP: <To Annotate LR and push NN>